**db_maker.py**

In [1]:
import os
import pandas as pd

In [2]:
image_dir = os.path.join("student_db")
data = []

In [3]:
for index, filename in enumerate(os.listdir(image_dir), start=1):
    if filename.endswith(".png"):
        new_name = filename.replace(" ", "_")
        os.rename(str(image_dir) + os.sep + filename,
                  str(image_dir)+os.sep + new_name)
        print(new_name)
        data.append([new_name, new_name, index])

AAMIR_KHAN.png
AJITH_KUMAR.png
ALLU_ARJUN.png
ANIRUDH.png
ANUSHKA_SHARMA.png
BECKY_G.png
DAVID_WARNER.png
DEEPIKA_PADUKONE.png
DHANUSH.png
DUA_LIPA.png
DULQUER_SALMAN.png
EMINEM.png
HARISH_KALYAN.png
IAN_SOMERHALDER.png
JENIFER_LOPEZ.png
JOS_BUTTLER.png
KAMAL_HASSAN.png
KANE_WILLIAMSON.png
KARTHI.png
KEERTHY_SURESH.png
MADONNNA_SEBASTIAN.png
MAHESH_BABU.png
MESSI.png
MODI.png
MOHANLAL.png
M_S_DHONI.png
NANI.png
NAYANTHARA.png
NINA_DOBREV.png
panda.png
PAUL_WESLEY.png
PITBULL.png
RAJINIKANTH.png
RANVEER_SINGH.png
RONALDO.png
SAMANTHA.png
SCARLETT_JOHANSSON.png
SELENA_GOMEZ.png
SHARUKH_KHAN.png
SHAWN_MENDES.png
STEVE_SMITH.png
SURESH_RAINA.png
SURYA.png
SUSHANT_SINGH.png
TAYLOR_SWIFT.png
VIJAY.png
VIJAY_DEVARAKONDA.png
VIKRAM.png


In [4]:
df = pd.DataFrame(data, columns=['name', 'image', 'roll_no'])
df.to_excel("student_db/people_db.xlsx")

**reader.py**

In [5]:
df = pd.read_excel(r'student_db/people_db.xlsx')

In [6]:
roll_record = {}
for key, row in df.iterrows():
    roll_no = row['roll_no']
    name = row['name']
    image_path = row['image']
    roll_record[roll_no] = name

In [7]:
roll_record

{1: 'AAMIR_KHAN.png',
 2: 'AJITH_KUMAR.png',
 3: 'ALLU_ARJUN.png',
 4: 'ANIRUDH.png',
 5: 'ANUSHKA_SHARMA.png',
 6: 'BECKY_G.png',
 7: 'DAVID_WARNER.png',
 8: 'DEEPIKA_PADUKONE.png',
 9: 'DHANUSH.png',
 10: 'DUA_LIPA.png',
 11: 'DULQUER_SALMAN.png',
 12: 'EMINEM.png',
 13: 'HARISH_KALYAN.png',
 14: 'IAN_SOMERHALDER.png',
 15: 'JENIFER_LOPEZ.png',
 16: 'JOS_BUTTLER.png',
 17: 'KAMAL_HASSAN.png',
 18: 'KANE_WILLIAMSON.png',
 19: 'KARTHI.png',
 20: 'KEERTHY_SURESH.png',
 21: 'MADONNNA_SEBASTIAN.png',
 22: 'MAHESH_BABU.png',
 23: 'MESSI.png',
 24: 'MODI.png',
 25: 'MOHANLAL.png',
 26: 'M_S_DHONI.png',
 27: 'NANI.png',
 28: 'NAYANTHARA.png',
 29: 'NINA_DOBREV.png',
 30: 'panda.png',
 31: 'PAUL_WESLEY.png',
 33: 'PITBULL.png',
 34: 'RAJINIKANTH.png',
 35: 'RANVEER_SINGH.png',
 36: 'RONALDO.png',
 37: 'SAMANTHA.png',
 38: 'SCARLETT_JOHANSSON.png',
 39: 'SELENA_GOMEZ.png',
 40: 'SHARUKH_KHAN.png',
 41: 'SHAWN_MENDES.png',
 42: 'STEVE_SMITH.png',
 43: 'SURESH_RAINA.png',
 44: 'SURYA.png',
 45: 

**main.py**

In [8]:
import face_recognition
import cv2
import numpy as np
import os
import pandas as pd
import time
from utils import append_df_to_excel

In [9]:
video_capture = cv2.VideoCapture(0)

In [10]:
known_face_encodings = []
known_face_roll_no = []
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
attendance_record = set([])
roll_record = {}

In [11]:
name_col = []
roll_no_col = []
time_col = []

In [12]:
df = pd.read_excel("student_db" + os.sep + "people_db.xlsx")

In [13]:
for key, row in df.iterrows():
    roll_no = row['roll_no']
    name = row['name']
    image_path = row['image']
    roll_record[roll_no] = name
    student_image = face_recognition.load_image_file(
        "student_db" + os.sep + image_path)
    student_face_encoding = face_recognition.face_encodings(student_image)[0]
    known_face_encodings.append(student_face_encoding)
    known_face_roll_no.append(roll_no)

In [14]:
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=1, fy=1)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(
            rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(
                known_face_encodings, face_encoding, tolerance=0.5)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_roll_no[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(
                known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                roll_no = known_face_roll_no[best_match_index]
                # add this to the log
                name = roll_record[roll_no]
                if roll_no not in attendance_record:
                    attendance_record.add(roll_no)
                    print(name, roll_no)
                    name_col.append(name)
                    roll_no_col.append(roll_no)
                    curr_time = time.localtime()
                    curr_clock = time.strftime("%H:%M:%S", curr_time)
                    time_col.append(curr_clock)

            face_names.append(name)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        # top *= 2
        # right *= 2
        # bottom *= 2
        # left *= 2

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35),
                      (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6),
                    font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

SCARLETT_JOHANSSON.png 38


In [19]:
data = {'Name': name_col, 'Roll No.': roll_no_col, 'Time': time_col}

curr_time = time.localtime()
curr_clock = time.strftime("%c", curr_time)
log_file_name = f"attendance_record {curr_clock}.xlsx"

# append_df_to_excel(log_file_name, data)


# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()